# MD Folding Simulation of the TRP Cage Protein with Heavy-H (Artemis)




In [5]:
cd /project/MDPF/trp_cage/folding_study

/project/RDS-FSC-MDPF-RW/trp_cage/folding_study


In [14]:
%%file nvt.job
#!/bin/bash
#PBS -N trp_cage_nvt
#PBS -P MDPF

#PBS -l select=2:ncpus=6:mpiprocs=1:mem=32GB:ngpus=0
#PBS -l walltime=0:20:0
#PBS -o trp_cage_nvt.log

module load intel-mpi 
module load gromacs/2016.2-intel-mpi-gpu
cd $PBS_O_WORKDIR
cat ${PBS_NODEFILE} > nodes.txt
 
mpirun -np 2 gmx_mpi mdrun -v -ntomp 6 -deffnm nvt 

Overwriting nvt.job


In [6]:
%ls -la

total 10689048
drwxr-sr-x 2 psol3692 RDS-FSC-MDPF-RW       12288 Mar 11 23:23 ./
drwxr-sr-x 3 psol3692 RDS-FSC-MDPF-RW        4096 Mar 11 00:18 ../
-rw------- 1 psol3692 RDS-FSC-MDPF-RW    64499712 Mar 11 01:34 core.10149
-rw------- 1 psol3692 RDS-FSC-MDPF-RW    73682944 Mar 11 01:30 core.6737
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW      980100 Mar 11 01:38 md_0_1.cpt
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW        8052 Mar 11 01:38 md_0_1.edr
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW     2811666 Mar 11 01:38 md_0_1.gro
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW         397 Mar 11 01:35 md_0_1.job
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW       30616 Mar 11 01:38 md_0_1.log
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW     1239700 Mar 11 01:35 md_0_1.tpr
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW    10758792 Mar 11 01:38 md_0_1.trr
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW     1634724 Mar 11 01:38 md_0_1.xtc
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW      980100 Mar 11 01:38 md_0_2.cpt
-rw-r--r-- 1 psol3692 RDS-FSC-MDPF-RW  

In [15]:
!qsub nvt.job

1365582.pbsserver


In [2]:
!qstat -u psol3692


pbsserver: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
1365601.pbsserv psol3692 normal   trp_cage_m  47979   4  48  256gb 48:00 R 20:10


In [9]:
!tail -n 12 nvt.log

 PME redist. X/F        2    6     100002      18.065        541.937   6.3
 PME spread/gather      2    6     100002      45.118       1353.543  15.8
 PME 3D-FFT             2    6     100002      12.744        382.308   4.5
 PME 3D-FFT Comm.       2    6     100002       9.432        282.962   3.3
 PME solve Elec         2    6      50001       1.243         37.300   0.4
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:     3432.756      286.063     1200.0
                 (ns/day)    (hour/ns)
Performance:       30.204        0.795
Finished mdrun on rank 0 Sat Mar 11 00:36:00 2017


In [11]:
%%file npt.job
#!/bin/bash
#PBS -N trp_cage_npt
#PBS -P MDPF

#PBS -l select=4:ncpus=6:mpiprocs=1:mem=32GB:ngpus=0
#PBS -l walltime=0:10:0
#PBS -o trp_cage_npt.log

module load intel-mpi 
module load gromacs/2016.2-intel-mpi-gpu
cd $PBS_O_WORKDIR
cat ${PBS_NODEFILE} > nodes.txt
 
mpirun -np 4 gmx_mpi mdrun -v -ntomp 6 -deffnm npt 

Writing npt.job


In [11]:
!qsub npt.job

qsub: User psol3692 is not a member of project RDS-FSC-MDPF-RW.


In [23]:
!tail -n 12 npt.log

 PME redist. X/F        4    6     100002       9.039        562.707   6.9
 PME spread/gather      4    6     100002      16.368       1018.982  12.5
 PME 3D-FFT             4    6     100002       4.572        284.622   3.5
 PME 3D-FFT Comm.       4    6     100002       5.824        362.555   4.4
 PME solve Elec         4    6      50001       0.640         39.866   0.5
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:     3148.444      131.185     2400.0
                 (ns/day)    (hour/ns)
Performance:       65.862        0.364
Finished mdrun on rank 0 Sat Mar 11 00:45:02 2017


In [117]:
%%file md.mdp
; Run parameters
integrator  = md        ; leap-frog integrator
nsteps      = 50000000  ; 2 * 50000000 = 100000 ps (100 ns)
dt          = 0.002     ; 2 fs

; Output control
nstxout     = 5000       ; save coordinates every 10.0 ps
nstvout     = 5000       ; save velocities every 10.0 ps
nstenergy   = 5000       ; save energies every 10.0 ps
nstlog      = 5000       ; update log file every 10.0 ps
nstxout-compressed  = 5000      ; save compressed coordinates every 10.0 ps
                                ; nstxout-compressed replaces nstxtcout
compressed-x-grps   = System    ; replaces xtc-grps

; Bond parameters
continuation            = yes       ; restarting acter NPT
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = all-bonds ; all bonds (even heavy atom-H bonds) constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy

; Neighborsearching
cutoff-scheme   = Verlet
ns_type         = grid      ; search neighboring grid cells
nstlist         = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb        = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw            = 1.0       ; short-range van der Waals cutoff (in nm)

; Electrostatics
coulombtype     = PME   ; Particle Mesh Ewald for long-range electrostatics
pme_order       = 4     ; cubic interpolation
fourierspacing  = 0.16  ; grid spacing for FFT

; Temperature coupling is on
tcoupl      = V-rescale             ; modified Berendsen thermostat
tc-grps     = Protein Non-Protein   ; two coupling groups - more accurate
tau_t       = 0.1  0.1              ; time constant, in ps
ref_t       = 325  325              ; reference temperature, one for each group, in K

; Pressure coupling is on
pcoupl              = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype          = isotropic             ; uniform scaling of box vectors
tau_p               = 2.0                   ; time constant, in ps
ref_p               = 1.0                   ; reference pressure, in bar
compressibility     = 4.5e-5                ; isothermal compressibility of water, bar^-1
refcoord_scaling    = com

; Periodic boundary conditions
pbc         = xyz       ; 3-D PBC

; Dispersion correction
DispCorr    = EnerPres  ; account for cut-off vdW scheme

; Velocity generation
gen_vel     = no       ; Velocity generation is off


Overwriting md.mdp


In [118]:
%%file md_1_0.job
#!/bin/bash
#PBS -N trp_cage_md_1_0
#PBS -P ISSSM

#PBS -l select=4:ncpus=12:mpiprocs=1:mem=64GB:ngpus=0
#PBS -l walltime=48:00:0
#PBS -o trp_cage_md_1_0.log

module load intel-mpi 
module load gromacs/2016.2-intel-mpi
cd $PBS_O_WORKDIR
cat ${PBS_NODEFILE} > nodes.txt

gmx_mpi grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_1_0.tpr
mpirun -np 4 gmx_mpi mdrun -v -ntomp 12 -deffnm md_1_0

Writing md_1_0.job


In [119]:
!qsub md_1_0.job

1365601.pbsserver


In [70]:
!tail -n 12 md_0_1.log

 PME redist. X/F        4    6     100002       9.821        589.275   6.9
 PME spread/gather      4    6     100002      18.665       1119.909  13.1
 PME 3D-FFT             4    6     100002       5.701        342.036   4.0
 PME 3D-FFT Comm.       4    6     100002       5.760        345.626   4.1
 PME solve Elec         4    6      50001       0.688         41.268   0.5
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:     3413.359      142.223     2400.0
                 (ns/day)    (hour/ns)
Performance:       60.751        0.395
Finished mdrun on rank 0 Sat Mar 11 01:38:13 2017


In [75]:
!tail -n 12 md_0_2.log

 PME redist. X/F        8    6     100002      23.137       2880.875  16.8
 PME spread/gather      8    6     100002      17.677       2200.961  12.9
 PME 3D-FFT             8    6     100002       4.236        527.402   3.1
 PME 3D-FFT Comm.       8    6     200004      14.413       1794.588  10.5
 PME solve Elec         8    6      50001       0.332         41.386   0.2
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:     6592.784      137.350     4800.0
                 (ns/day)    (hour/ns)
Performance:       62.906        0.382
Finished mdrun on rank 0 Sat Mar 11 01:38:36 2017


In [82]:
!tail -n 12 md_0_3.log

 PME redist. X/F       16    6     100002      30.219       7525.261  26.7
 PME spread/gather     16    6     100002      15.223       3790.781  13.5
 PME 3D-FFT            16    6     100002       3.535        880.253   3.1
 PME 3D-FFT Comm.      16    6     200004      12.856       3201.360  11.4
 PME solve Elec        16    6      50001       0.180         44.915   0.2
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:    10852.383      113.046     9600.0
                 (ns/day)    (hour/ns)
Performance:       76.431        0.314
Finished mdrun on rank 0 Sat Mar 11 01:40:43 2017


In [90]:
!tail -n 12 md_0_4.log

 PME redist. X/F        4   12     100002       9.857       1227.348  10.4
 PME spread/gather      4   12     100002      13.171       1639.985  13.9
 PME 3D-FFT             4   12     100002       3.748        466.699   4.0
 PME 3D-FFT Comm.       4   12     100002       6.140        764.520   6.5
 PME solve Elec         4   12      50001       0.327         40.758   0.3
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:     4538.585       94.554     4800.0
                 (ns/day)    (hour/ns)
Performance:       91.378        0.263
Finished mdrun on rank 0 Sat Mar 11 01:42:32 2017


In [104]:
!tail -n 12 md_0_5.log

 PME redist. X/F        2   24     100002      16.384       2039.953  12.5
 PME spread/gather      2   24     100002      17.874       2225.578  13.7
 PME 3D-FFT             2   24     100002       6.109        760.586   4.7
 PME 3D-FFT Comm.       2   24     100002       9.201       1145.598   7.0
 PME solve Elec         2   24      50001       0.332         41.322   0.3
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:     6281.003      130.854     4800.0
                 (ns/day)    (hour/ns)
Performance:       66.029        0.363
Finished mdrun on rank 0 Sat Mar 11 01:46:03 2017


In [7]:
!tail -n 12 md_1_0.log

       43400000    86800.00000

   Energies (kJ/mol)
          Angle    Proper Dih. Ryckaert-Bell.          LJ-14     Coulomb-14
    6.42949e+02    3.59351e+01    2.90648e+02    4.08731e+02    1.75765e+03
        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip.      Potential
    1.15476e+05   -4.93838e+03   -7.31006e+05    3.64844e+03   -6.13684e+05
    Kinetic En.   Total Energy    Temperature Pres. DC (bar) Pressure (bar)
    1.10572e+05   -5.03112e+05    3.26398e+02   -1.98340e+02   -1.00706e+02
   Constr. rmsd
    6.65874e-05



In [115]:
!rm \#*

In [130]:
!qstat -u psol3692


pbsserver: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
1365597.pbsserv psol3692 gpu      trp_cage_m    --    1  12   64gb 00:15 Q   -- 
1365598.pbsserv psol3692 gpu      trp_cage_m    --    2  24  128gb 00:15 Q   -- 
1365599.pbsserv psol3692 gpu      trp_cage_m    --    3  36  192gb 00:15 Q   -- 
1365600.pbsserv psol3692 gpu      trp_cage_m    --    4  48  256gb 00:15 Q   -- 
1365601.pbsserv psol3692 normal   trp_cage_m  47979   4  48  256gb 48:00 R 00:01


In [129]:
!qalter 1365600 -l walltime=00:15:00